# **Exercise: Accelerating Jacobi's Method for Eigenvalue Computation**

Jacobi's method iteratively reduces off-diagonal elements of a symmetric matrix to zero. However, due to numerical effects, previously eliminated off-diagonal elements may reappear, requiring many iterations for full convergence.

To accelerate convergence, we first reduce the matrix to tridiagonal form before applying Jacobi's method. This reduction preserves certain off-diagonal zeros throughout the iterations, improving efficiency.

## **Tasks**

### (i) Implement Tridiagonal Reduction
- Write a Python script that first reduces a given symmetric matrix to tridiagonal form before applying Jacobi's method. Consider implementing Householder reduction.
- Verify that your script correctly maintains the structure of a tridiagonal matrix throughout the iterations.

### (ii) Compare Performance Using Iteration Count
Instead of measuring execution time, compare the **number of iterations required for convergence**, as this provides a more reliable efficiency metric.

1. Implement both:
   - The **standard Jacobi method**.
   - The **tridiagonalized Jacobi method** (applying tridiagonal reduction first).
2. Set the convergence criterion discussed in class (Jacobi's notes, *Section 2.2.3*)

3. Generate a sequence of random symmetric matrices of size $N \times N$ and test both scripts. Count the number of iterations required for each method to meet the convergence criteria.

   ```python
   import numpy as np

   def generate_symmetric_matrix(N):
       A = np.random.rand(N, N)
       return (A + A.T) / 2  # Ensure symmetry

   N_values = [4, 8, 16, 32, 64, 128, 256, ...]
   matrices = [generate_symmetric_matrix(N) for N in N_values]
   ```

### (iii) Analyze and Discuss Results
- Compare how the number of iterations scales with $N$.
- Does tridiagonal reduction significantly speed up convergence?
- How does the efficiency improvement change for large matrices?
- Verify eigenvalues using `numpy.linalg.eigh()`.
- Extend the analysis by plotting the decay of the sum of squared off-diagonal elements over iterations.

## **Implementation**
We define some auxiliary functions to implement the optimized version of the Jacobi Method using householder matrices.

In [ ]:
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

class MatrixTrasformations():
    @staticmethod
    def householder(u):
        """Construct the householder matrix given a vector u: nx1"""
        n = u.shape[0]
        u = u/np.linalg.norm(u)
        return np.eye(n) - 2*np.outer(u, u)

    @staticmethod
    def to_upper_hessenberg(A: np.ndarray) -> np.ndarray:
        """Transforms a matrix into upper Hessenberg form through Householder reflexions"""
        T = A.copy().astype(float)
        n = A.shape[0]
        for i in range(n-2):
            x = T[i+1:, i]
            Hx = np.zeros(n-i-1) 
            Hx[0] = -np.sign(x[0]) * np.linalg.norm(x)
            H = np.eye(n)
            H[i+1:, i+1:] = MatrixTrasformations.householder(x-Hx)
            T = H @ T @ H
        return T

# Create an alias for MatrixTrasformations class
mt = MatrixTrasformations

In [64]:
N = 4
A = np.random.rand(N, N)
print(A)
print(mt.to_upper_hessenberg(A))

[[0.659 0.079 0.757 0.978]
 [0.367 0.602 0.678 0.210]
 [0.990 0.996 0.127 0.453]
 [0.613 0.265 0.249 0.030]]
[[0.659 -1.129 -0.328 0.393]
 [-1.221 0.911 -0.861 -0.269]
 [-0.000 -0.595 0.168 -0.178]
 [-0.000 -0.000 -0.232 -0.321]]


In [ ]:
class JacobiMethod():
    @staticmethod
    def sum(D, n):
        return np.sqrt(np.sum(np.diag(D)**2)/n)

def jacobi_method(A: np.ndarray, tol=10e-4):
    """
    Jacobi method for finding eigenvalues and eigenvectors of a matrix A.

    Args:
    - A (numpy.ndarray): A square matrix (nxn).
    - tol (float): The tolerance for convergence.

    Returns:
    - V (numpy.ndarray): Matrix of eigenvectors.
    - D (numpy.ndarray): Diagonal matrix of eigenvalues.
    - it (int): Number of iterations.
    """
    # Initialize V, D, and parameters:
    n = A.shape[0]
    D = A.copy().astype(float)
    V = np.eye(n)

    # Calculate row p and column q of the off-diagonal element of greatest magnitude in D:
    it = 1
    matrix = np.tril(np.abs(D - np.diag(np.diag(D))),-1)
    p, q = np.unravel_index(np.argmax(matrix), matrix.shape)
    while np.abs(D[p, q]) > tol*JacobiMethod.sum(D, n):
        # Compute rotation parameters
        beta = (D[q, q] - D[p, p]) / (2 * D[p, q])
        t = np.sign(beta) / (np.abs(beta) + np.sqrt(beta**2 + 1))
        c = 1 / np.sqrt(1 + t**2)
        s = c * t
        R = np.array([[c, s], [-s, c]])

        # Zero out D_pq and D_qp
        D[[p, q], :] = np.dot(R.T, D[[p, q], :])
        D[:, [p, q]] = np.dot(D[:, [p, q]], R)
        V[:, [p, q]] = np.dot(V[:, [p, q]], R)

        # Update row p and column q of the off-diagonal element of greatest magnitude in D for the next iteration:
        it += 1
        matrix = np.tril(np.abs(D - np.diag(np.diag(D))),-1)
        p, q = np.unravel_index(np.argmax(matrix), matrix.shape)

    return np.diag(D), V, it

def tri_jacobi_method(A: np.ndarray, tol=10e-4):
    A = mt.to_upper_hessenberg(A)
    return jacobi_method(A, tol)

We generate a sample of symmetric matrices

In [66]:
def generate_symmetric_matrix(N):
    A = np.random.rand(N, N)
    return (A + A.T) / 2  # Ensure symmetry

N_values = [4, 8, 16, 32, 64, 128, 256]
matrices = [generate_symmetric_matrix(N) for N in N_values]

And compare the efficiency from each algorithm

In [74]:
jm_results = []
tjm_results = []
for n, A in zip(N_values, matrices):
    jm_results.append(jacobi_method(A))
    tjm_results.append(tri_jacobi_method(A))

KeyboardInterrupt: 

In [75]:
for i, j in zip(jm_results, tjm_results):
    print(i[2], j[2])

11 13
53 45
223 174
922 483
3472 1994
12504 5350
